## Analyzing multiple scATAC-seq datasets for mouse brain using Catactor
* Risa K. Kawaguchi, et al (to be published).
* 2020.12.xx



## 1. Download and preprocess datasets
* From GEO (gene expression omnibus)
 * GSE100033
 * GSE111586
 * GSE123576
 * GSE126074 (SNARE-seq)
 * GSE127257
 * GSE130399 (Paired-seq)
* From BICCN database
 * BICCN (SnapATAC objects)
 * BICCN SMART-seq v2 data

## Assumed data structure
### Raw matrix data
raw_data/
* GSE100033
* GSE111586
* GSE123576
* GSE126074
* GSE127257
* GSE130399
* BICCN
* BICCN_rna

### Processed data
mat_data/
* GSE100033
* GSE111586
* GSE123576
* GSE126074
* GSE127257
* GSE1303990 <- Adult dataset 
* GSE1303991 <- Fetal dataset (unused)
* BICCN
* BICCN_rna

## 2. Preprocess downloaded datasets to produce sparse matrix filesfor Catactor
Place original data to a specific directory and apply preprocessing to make matrices in the same format.


In [5]:
# Construct matrix data
import os
import subprocess
script_dir = "../script/data_processing"
input_dir = "../raw_data"
output_dir = "../mat_data"

GSE_list = ['GSE100033', 'GSE111586', 'GSE123576', 'GSE126074', 'GSE127257', 'GSE130399', 'BICCN_rna']
GSE_dir = ['GSE100033', 'GSE111586', 'GSE123576', 'GSE126074', 'GSE127257', 'GSE1303990', 'BICCN_rna']
for gse, dir in zip(GSE_list, GSE_dir):
    print(gse+' processing...')
    arg_list = ['python', os.path.join(script_dir, 'data_preprocess.py'), gse, os.path.join(input_dir, gse), os.path.join(output_dir, dir)]
    print(' '.join(arg_list))
    subprocess.run(arg_list)


GSE100033 processing...
python ../script/data_processing/data_preprocess.py GSE100033 ../raw_data/GSE100033 ../mat_data/GSE100033
GSE111586 processing...
python ../script/data_processing/data_preprocess.py GSE111586 ../raw_data/GSE111586 ../mat_data/GSE111586
GSE123576 processing...
python ../script/data_processing/data_preprocess.py GSE123576 ../raw_data/GSE123576 ../mat_data/GSE123576
GSE126074 processing...
python ../script/data_processing/data_preprocess.py GSE126074 ../raw_data/GSE126074 ../mat_data/GSE126074
GSE127257 processing...
python ../script/data_processing/data_preprocess.py GSE127257 ../raw_data/GSE127257 ../mat_data/GSE127257
GSE130399 processing...
python ../script/data_processing/data_preprocess.py GSE130399 ../raw_data/GSE130399 ../mat_data/GSE1303990
BICCN_rna processing...
python ../script/data_processing/data_preprocess.py BICCN_rna ../raw_data/BICCN_rna ../mat_data/BICCN_rna


In [9]:
# BICCN scATAC-seq data (SnapATAC object) preprocessing
script_dir = "../script/data_processing"
input_dir = "../raw_data/BICCN"
output_dir = "../mat_data/BICCN"
# ulimit -s 100000 #if needed

arg_list = ['Rscript', os.path.join(script_dir, 'snap_atac_to_text.R'), input_dir, output_dir]
print(' '.join(arg_list))
# subprocess.run(arg_list)

['Rscript', '../script/data_processing/snap_atac_to_text.R', '../raw_data/BICCN', '../mat_data/BICCN']
['python', '../script/data_processing/biccn_data_reading.py', '../mat_data/BICCN', '../mat_data/BICCN']


In [ ]:
arg_list = ['python', os.path.join(script_dir, 'biccn_data_reading.py'), output_dir, output_dir]
print(' '.join(arg_list))
subprocess.run(arg_list)

## 3. Annotate each genomic bin with the closest genes
* Add global genomic bin ids
* Add closest gene ids


In [6]:
# Add a global id for each bin-size
import glob
import os
for dir in ['GSE100033', 'GSE111586', 'GSE123576', 'GSE126074', 'GSE127257', 'GSE1303990', 'BICCN_rna', 'BICCN']:
    for fname in glob.glob(os.path.join("../mat_data/", dir, "*bin*sv")):
        if 'with_bins' in fname:
            continue
        if 'gene.csv' in fname:
            continue
        arg_list = ['python', '../src/Catactor', 'column_annotation', fname]
        print(' '.join(arg_list))
        subprocess.run(arg_list)


python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_e11.5.csv
python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_e12.5.csv
python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_e13.5.csv
python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_e14.5.csv
python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_e15.5.csv
python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_e16.5.csv
python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_p0.csv
python ../src/Catactor column_annotation ../mat_data/GSE100033/GSE100033_bin_ng_p56.csv
python ../src/Catactor column_annotation ../mat_data/GSE111586/GSE111586_bin_ng_Prefrontal.csv
python ../src/Catactor column_annotation ../mat_data/GSE111586/GSE111586_bin_ng_Wholebrain1.csv
python ../src/Catactor column_annotation ../mat_data/GSE111586/GSE111586_bin_ng_Wholebrain2.cs

In [7]:
# Associate the closest gene ids
import glob
import os
for dir in ['GSE100033', 'GSE111586', 'GSE123576', 'GSE126074', 'GSE127257', 'GSE1303990', 'BICCN_rna', 'BICCN']:
    for fname in glob.glob(os.path.join("../mat_data/", dir, "*bin*sv")):
        if 'with_bins' not in fname:
            continue
        if 'gene.csv' in fname:
            continue
        arg_list = ['Rscript', '../script/data_annotation/annotation_metadata.R', 'promoter', fname]
        print(' '.join(arg_list))
        #subprocess.run(arg_list)


Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_e11.5_with_bins.csv
Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_e12.5_with_bins.csv
Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_e13.5_with_bins.csv
Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_e14.5_with_bins.csv
Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_e15.5_with_bins.csv
Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_e16.5_with_bins.csv
Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_p0_with_bins.csv
Rscript ../script/data_annotation/annotation_metadata.R promoter ../mat_data/GSE100033/GSE100033_bin_ng_p56_with_bins.csv
Rscript ../sc